In [18]:
#Libraries to import

import statsapi
import pandas as pd
import json
import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import colors
from openpyxl.styles import Font, Color, Alignment
import urllib.request
from IPython.display import IFrame

from openpyxl.drawing.image import Image
import PIL
import io
import urllib3

In [16]:
#Input to select Player and stats type for Baseball Card

player_last_name = input('Last Name of Player for Player Card: ')
player_first_name = input('First Name of Player for Player Card: ')
stats_type = input('Stats to display in the card (hitting or pitching): ')

#Lookup Player
player = statsapi.lookup_player(player_first_name+" "+player_last_name)



#Get Player Headshot
player_id = player[0]['id']

IFrame(f"https://img.mlbstatic.com/mlb-photos/image/upload/w_248,q_100/v1/people/{player_id}/headshot/83/current", 700, 300)

Last Name of Player for Player Card: Springs
First Name of Player for Player Card: Jeffrey
Stats to display in the card (hitting or pitching): pitching


In [3]:
#Get Player Biographical Information

player_bio = statsapi.get('people', {'personIds': player[0]["id"]})

bio_df = pd.json_normalize(player_bio['people'])




#Get Player Full Name
print('Name')
player_full_name = print(player[0]['nameFirstLast'])

print('')

#Get Player Uniform Number
print('Number')
uniform_number = print(bio_df._get_value(0,'primaryNumber'))

print('')

#Get Player's Primary Position
print('Position')
primary_position = print(bio_df._get_value(0,'primaryPosition.abbreviation'))

print('')

#Get Player's Hitting Handedness
print('Bats')
bats = print(bio_df._get_value(0,'batSide.code'))

print('')

#Get Player's Pitching/Throwing Handedness
print('Throws')
throws = print(bio_df._get_value(0,'pitchHand.code'))

print('')

#Get Player's Birthplace 
print('Place of Birth')
birth_country = bio_df._get_value(0,'birthCountry')
birth_city = bio_df._get_value(0,'birthCity')
if birth_country == 'USA':
    birth_state = bio_df._get_value(0,'birthStateProvince')
    birthplace = birth_city + ', ' + birth_state + ', ' + birth_country
    print(birthplace)
else:
    birthplace = birth_city + ', ' + birth_country
    print(birthplace)

print('')

#Get Player's Date of Birth
print('Date of Birth')
birthdate = print(bio_df._get_value(0,'birthDate'))

print('')

#Get Player's Height
print('Height')
height = print(bio_df._get_value(0,'height'))

print('')

#Get Player's Weight
print('Weight')
weight = print(bio_df._get_value(0,'weight'))

print('')

#Get Player's Current Team
print('Team')
player_team = statsapi.lookup_team(player[0]['currentTeam']['id'])
player_team_name = print(player_team[0]['name'])


print('')

#Get Player's MLB Debut Date
print('MLB Debut')
player_mlb_debut = print(player[0]['mlbDebutDate'])

Name
Jeffrey Springs
Number
59

Position
P

Bats
L

Throws
L

Place of Birth
Belmont, NC, USA

Date of Birth
1992-09-20

Height
6' 3"

Weight
218

Team
Tampa Bay Rays

MLB Debut
2018-07-31


In [4]:
# Get Player's Awards Won

## Build parameters for each award
alroy_params = {
    'awardId':'ALROY',
    'sportId':1,
    'recipients':True
}

alroy = pd.json_normalize(statsapi.get('awards', alroy_params)['awards'])

nlroy_params = {
    'awardId':'NLROY',
    'sportId':1,
    'recipients':True
}

nlroy = pd.json_normalize(statsapi.get('awards', nlroy_params)['awards'])

algg_params = {
    'awardId':'ALGG',
    'sportId':1,
    'recipients':True
}

algg = pd.json_normalize(statsapi.get('awards', algg_params)['awards'])

nlgg_params = {
    'awardId':'NLGG',
    'sportId':1,
    'recipients':True
}

nlgg = pd.json_normalize(statsapi.get('awards', nlgg_params)['awards'])

alas_params = {
    'awardId':'ALAS',
    'sportId':1,
    'recipients':True
}

alas = pd.json_normalize(statsapi.get('awards', alas_params)['awards'])

nlas_params = {
    'awardId':'NLAS',
    'sportId':1,
    'recipients':True
}

nlas = pd.json_normalize(statsapi.get('awards', nlas_params)['awards'])

almvp_params = {
    'awardId':'ALMVP',
    'sportId':1,
    'recipients':True
}

almvp = pd.json_normalize(statsapi.get('awards', almvp_params)['awards'])

nlmvp_params = {
    'awardId':'NLMVP',
    'sportId':1,
    'recipients':True
}

nlmvp = pd.json_normalize(statsapi.get('awards', nlmvp_params)['awards'])

alss_params = {
    'awardId':'ALSS',
    'sportId':1,
    'recipients':True
}

alss = pd.json_normalize(statsapi.get('awards', alss_params)['awards'])

nlss_params = {
    'awardId':'NLSS',
    'sportId':1,
    'recipients':True
}

nlss = pd.json_normalize(statsapi.get('awards', nlss_params)['awards'])

alcy_params = {
    'awardId':'ALCY',
    'sportId':1,
    'recipients':True
}

alcy = pd.json_normalize(statsapi.get('awards', alcy_params)['awards'])


nlcy_params = {
    'awardId':'NLCY',
    'sportId':1,
    'recipients':True
}

nlcy = pd.json_normalize(statsapi.get('awards', nlcy_params)['awards'])

#Build Awards receipient Dataframe
awards = pd.concat([alroy[['player.id','season','name']],
                    nlroy[['player.id','season','name']],
                    algg[['player.id','season','name']],
                    nlgg[['player.id','season','name']],
                    alas[['player.id','season','name']],
                    nlas[['player.id','season','name']],
                    almvp[['player.id','season','name']],
                    nlmvp[['player.id','season','name']],
                    alss[['player.id','season','name']],
                    nlss[['player.id','season','name']],
                    alcy[['player.id','season','name']],
                    nlcy[['player.id','season','name']]
                   ])

#Cross-Reference awards recipients dataframe with the selected player
player_awards = awards.loc[awards['player.id'] == player_id]


#Build string from dataframe for awards won
player_awards = player_awards[['season','name']].sort_values(by=['season']).style.hide_index()

player_awards_df = player_awards.data

player_awards_df.rename(columns = {'season': 'Season', 'name': 'Award'}, inplace = True)

player_awards_str = player_awards_df.to_string(index = False)

if pd.DataFrame(player_awards_df).empty:
    print("No Awards won so far")

else:
    print(player_awards_str)

No Awards won so far


In [8]:
#Get Player's Season Stats

## Create parameters for the stats and person endpoint
### 'group' will provide what stat type we need to look for (pitching, hitting)
group = stats_type
type = 'yearByYear'
params = {
    "personId": player[0]['id'],
    "hydrate": "stats(group=" + group + ",type=" + type + ")",
}

##Make an API call for the required endpoints using the parameters outlined above
try:
    
    r = statsapi.get("person", params)['people']

    if stats_type == 'hitting':
    
        player_stats_hitting = pd.DataFrame(columns = ['Season','Games','Plate App','At Bats','BB','SO','Hits',
                                                       'Doubles','Triples','HRs','AVG','OBP','SLG','OPS','wOBA','K%','BB%'])



        for item in r[0]['stats']:
    
            for i in range(0, int(len(r[0]['stats'][0]['splits']))):
                season_played = item['splits'][i]['season']
                #team_played = item['splits'][i]['team']['name']
                games_played = item['splits'][i]['stat']['gamesPlayed']
                plate_app = item['splits'][i]['stat']['plateAppearances']
                at_bats = item['splits'][i]['stat']['atBats']
                bat_k = item['splits'][i]['stat']['strikeOuts']
                bat_bb = item['splits'][i]['stat']['baseOnBalls']
                bat_hbp = item['splits'][i]['stat']['hitByPitch']
                int_bb = item['splits'][i]['stat']['intentionalWalks']
                sac_fly = item['splits'][i]['stat']['sacFlies']
                hits = item['splits'][i]['stat']['hits']
                doubles = item['splits'][i]['stat']['doubles']
                triples = item['splits'][i]['stat']['triples']
                home_runs = item['splits'][i]['stat']['homeRuns']
                bat_avg = item['splits'][i]['stat']['avg']
                bat_obp = item['splits'][i]['stat']['obp']
                bat_slg = item['splits'][i]['stat']['slg']
                bat_ops = float(bat_obp)+float(bat_slg)
                try: bat_woba = format(float((float(.69*bat_bb)+float(.72*bat_hbp)+
                              float(.87*(float(hits)-float(doubles)-float(triples)-float(home_runs)))+
                                              float(1.22*float(doubles))+
                                    float(1.53*float(triples))+float(1.94*float(home_runs)))/
                             (float(at_bats)+float(bat_bb)+float(sac_fly)+float(bat_hbp)-float(int_bb))),'.3f')
                except:
                    bat_woba = 0
        
                try: bb_rate = "{:.0%}".format(float(bat_bb)/float(plate_app))
        
                except:
                    bb_rate = 0
            
                try: k_rate = "{:.0%}".format(float(bat_k)/float(plate_app))
        
                except:
                    k_rate = 0
        
                player_stats_hitting.loc[i] = [season_played, #team_played,
                                       games_played, plate_app, at_bats, bat_bb,
                                       bat_k, hits, doubles, triples, home_runs, bat_avg, bat_obp, bat_slg,
                                       bat_ops, bat_woba, k_rate, bb_rate]
            print(player_stats_hitting)

    else:
        player_stats_pitching = pd.DataFrame(columns = ['Season','Games','IP','BF','Wins',
                                                        'Losses','Saves','Save Opp','SO','BB','HBP','Int BB','ER',
                                                        'ERA','RA','WHIP','GB/FB','Pitch/Inn', 'HR', 'K/BB', 'K/9', 'BB/9',
                                                        'HR/9', 'Opp OPS'])



        for item in r[0]['stats']:
    
            for i in range(0, int(len(r[0]['stats'][0]['splits']))):
                season_played = item['splits'][i]['season']
                #team_played = item['splits'][i]['team']['name']
                games_pitched = item['splits'][i]['stat']['gamesPitched']
                innings_pitched = item['splits'][i]['stat']['inningsPitched']
                batters_faced = item['splits'][i]['stat']['battersFaced']
                wins = item['splits'][i]['stat']['wins']
                losses = item['splits'][i]['stat']['losses']
                saves = item['splits'][i]['stat']['saves']
                save_opp = item['splits'][i]['stat']['saveOpportunities']    
                pitch_k = item['splits'][i]['stat']['strikeOuts']
                pitch_bb = item['splits'][i]['stat']['baseOnBalls']
                pitch_hbp = item['splits'][i]['stat']['hitBatsmen']
                int_bb = item['splits'][i]['stat']['intentionalWalks']
                earned_runs = item['splits'][i]['stat']['earnedRuns']
                era = item['splits'][i]['stat']['era']
                runs_per9 = item['splits'][i]['stat']['runsScoredPer9']
                whip = item['splits'][i]['stat']['whip']
                gb_fb = item['splits'][i]['stat']['groundOutsToAirouts']
                pitch_per_ip = item['splits'][i]['stat']['pitchesPerInning']
                home_runs = item['splits'][i]['stat']['homeRuns']
                k_to_bb = item['splits'][i]['stat']['strikeoutWalkRatio']
                k_per9 = item['splits'][i]['stat']['strikeoutsPer9Inn']
                bb_per9 = item['splits'][i]['stat']['walksPer9Inn']
                hr_per9 = item['splits'][i]['stat']['homeRunsPer9']
                pitch_obp = item['splits'][i]['stat']['obp']
                pitch_slg = item['splits'][i]['stat']['slg']
                pitch_ops = float(pitch_obp)+float(pitch_slg)
                
                player_stats_pitching.loc[i] = [season_played, games_pitched, innings_pitched, batters_faced, wins, losses,
                                                saves, save_opp, pitch_k, pitch_bb, pitch_hbp, int_bb, earned_runs, era,
                                                runs_per9, whip, gb_fb, pitch_per_ip, home_runs, k_to_bb, k_per9, bb_per9,
                                                hr_per9, pitch_ops]


        print(player_stats_pitching)
        
except:
    print('No available stats for that category for the selected player')

  Season Games Pitched Innings Pitched Batters Faced Wins Losses Saves  \
0   2018            18            32.0           141    1      1     0   
1   2019            25            32.1           155    4      1     0   
2   2020            16            20.1            99    0      2     0   
3   2021            43            44.2           179    5      1     2   
4   2022            33           135.1           549    9      5     0   

  Save Opp   SO  BB  ...    RA  WHIP GB/FB Pitch per Inn  HR  K/BB    K/9  \
0        1   31  14  ...  3.94  1.44  0.66         17.56   4  2.21   8.72   
1        0   32  23  ...  6.40  1.89  0.42         20.57   4  1.39   8.91   
2        1   28   7  ...  7.97  1.82  0.83         19.92   5  4.00  12.39   
3        4   63  14  ...  4.23  1.10  0.68         16.81   9  4.50  12.69   
4        0  144  31  ...  2.79  1.07  0.75         15.88  14  4.65   9.58   

   BB/9  HR/9 Opp OPS  
0  3.94  1.13   0.744  
1  6.40  1.11   0.884  
2  3.10  2.21   0.97

In [7]:
#Get Player's Career Stats

## Create parameters for the stats and person endpoint
### 'group' will provide what stat type we need to look for (pitching, hitting)
group = stats_type
type = 'career'
params = {
    "personId": player[0]['id'],
    "hydrate": "stats(group=" + group + ",type=" + type + ")",
}

##Make an API call for the required endpoints using the parameters outlined above
  
r = statsapi.get("person", params)['people']

if stats_type == 'hitting':
    player_career_hitting = pd.DataFrame(columns = ['Career','Games','PA','At Bats','BB','SO','Hits',
                                                    '2B','3B','HRs','AVG','OBP','SLG','OPS','wOBA','K%','BB%'])



    for item in r[0]['stats']:
        season_played = item['type']['displayName']
        #team_played = item['splits'][i]['team']['name']
        games_played = item['splits'][0]['stat']['gamesPlayed']
        plate_app = item['splits'][0]['stat']['plateAppearances']
        at_bats = item['splits'][0]['stat']['atBats']
        bat_k = item['splits'][0]['stat']['strikeOuts']
        bat_bb = item['splits'][0]['stat']['baseOnBalls']
        bat_hbp = item['splits'][0]['stat']['hitByPitch']
        int_bb = item['splits'][0]['stat']['intentionalWalks']
        sac_fly = item['splits'][0]['stat']['sacFlies']
        hits = item['splits'][0]['stat']['hits']
        doubles = item['splits'][0]['stat']['doubles']
        triples = item['splits'][0]['stat']['triples']
        home_runs = item['splits'][0]['stat']['homeRuns']
        bat_avg = item['splits'][0]['stat']['avg']
        bat_obp = item['splits'][0]['stat']['obp']
        bat_slg = item['splits'][0]['stat']['slg']
        bat_ops = float(bat_obp)+float(bat_slg)
        try: bat_woba = format(float((float(.69*bat_bb)+float(.72*bat_hbp)+
                                      float(.87*(float(hits)-float(doubles)-float(triples)-float(home_runs)))+
                                      float(1.22*float(doubles))+
                                      float(1.53*float(triples))+float(1.94*float(home_runs)))/
                                     (float(at_bats)+float(bat_bb)+float(sac_fly)+float(bat_hbp)-float(int_bb))),'.3f')
        except:
            bat_woba = 0
        
        try:
            bb_rate = "{:.0%}".format(float(bat_bb)/float(plate_app))
        
        except:
            bb_rate = 0
            
        try:
            k_rate = "{:.0%}".format(float(bat_k)/float(plate_app))
        
        except:
            k_rate = 0
        
        player_career_hitting.loc[i] = [season_played, #team_played,
                                       games_played, plate_app, at_bats, bat_bb,bat_k, hits, doubles,
                                       triples, home_runs, bat_avg, bat_obp, bat_slg, bat_ops,
                                       bat_woba, k_rate, bb_rate]
        print(player_career_hitting)

else:
    player_career_pitching = pd.DataFrame(columns = ['Career', 'Games','IP','BF','Wins',
                                                     'Losses','Saves','Save Opp','SO','BB','HBP','Int BB','ER',
                                                     'ERA','RA','WHIP','GB/FB','Pitch/Inn', 'HR', 'K/BB', 'K/9', 'BB/9',
                                                     'HR/9', 'Opp OPS'])



    for item in r[0]['stats']:
        season_played = item['type']['displayName']
        #team_played = item['splits'][i]['team']['name']
        games_pitched = item['splits'][0]['stat']['gamesPitched']
        innings_pitched = item['splits'][0]['stat']['inningsPitched']
        batters_faced = item['splits'][0]['stat']['battersFaced']
        wins = item['splits'][0]['stat']['wins']
        losses = item['splits'][0]['stat']['losses']
        saves = item['splits'][0]['stat']['saves']
        save_opp = item['splits'][0]['stat']['saveOpportunities']    
        pitch_k = item['splits'][0]['stat']['strikeOuts']
        pitch_bb = item['splits'][0]['stat']['baseOnBalls']
        pitch_hbp = item['splits'][0]['stat']['hitBatsmen']
        int_bb = item['splits'][0]['stat']['intentionalWalks']
        earned_runs = item['splits'][0]['stat']['earnedRuns']
        era = item['splits'][0]['stat']['era']
        runs_per9 = item['splits'][0]['stat']['runsScoredPer9']
        whip = item['splits'][0]['stat']['whip']
        gb_fb = item['splits'][0]['stat']['groundOutsToAirouts']
        pitch_per_ip = item['splits'][0]['stat']['pitchesPerInning']
        home_runs = item['splits'][0]['stat']['homeRuns']
        k_to_bb = item['splits'][0]['stat']['strikeoutWalkRatio']
        k_per9 = item['splits'][0]['stat']['strikeoutsPer9Inn']
        bb_per9 = item['splits'][0]['stat']['walksPer9Inn']
        hr_per9 = item['splits'][0]['stat']['homeRunsPer9']
        pitch_obp = item['splits'][0]['stat']['obp']
        pitch_slg = item['splits'][0]['stat']['slg']
        pitch_ops = float(pitch_obp)+float(pitch_slg)
                
        player_career_pitching.loc[i] = [season_played, games_pitched, innings_pitched, batters_faced, wins, losses,
                                        saves, save_opp, pitch_k, pitch_bb, pitch_hbp, int_bb, earned_runs, era,
                                        runs_per9, whip, gb_fb, pitch_per_ip, home_runs, k_to_bb, k_per9, bb_per9,
                                        hr_per9, pitch_ops]


    print(player_career_pitching)
        

   Career Games     IP    BF Wins Losses Saves Save Opp   SO  BB  ...    RA  \
0  career   135  264.2  1123   19     10     2        6  298  89  ...  4.01   

   WHIP GB/FB Pitch/Inn  HR  K/BB    K/9  BB/9  HR/9 Opp OPS  
0  1.28  0.68     17.12  36  3.35  10.13  3.03  1.22   0.717  

[1 rows x 24 columns]


In [10]:
#Get Draft information if player participated in the draft
params = {
    "personId": player[0]['id'],
    "hydrate": "draft(year=True)"
}



dr = statsapi.get("person", params)['people']

try:
    print('Year Drafted')
    draft_year = print(dr[0]['draftYear'])
    print('')
    print('Round Selected')
    draft_round = print(dr[0]['drafts'][0]['pickRound'])
    print('')
    print('Pick Selected')
    draft_round_pick = print(dr[0]['drafts'][0]['roundPickNumber'])
    print('')
    print('Overall Pick')
    draft_overall_pick = print(dr[0]['drafts'][0]['pickNumber'])
    print('')
    print('School Name')
    draft_school = print(dr[0]['drafts'][0]['school']['name'])
    
except:
    print('Did Not Participate in Draft')


Year Drafted
2015

Round Selected
30

Pick Selected
3

Overall Pick
888

School Name
Appalachian State


In [22]:
#Create workbook for the baseball card output

wb = Workbook()

ws = wb.active

ws.title = 'Card'


#Add Headshot
r = 1
http = urllib3.PoolManager()
r = http.request('GET', f"https://img.mlbstatic.com/mlb-photos/image/upload/w_248,q_100/v1/people/{player_id}/headshot/83/current")
image_file = io.BytesIO(r.data)
img = Image(image_file)
img.height = 263.04
img.width = 214.08
ws.add_image(img, 'J1')


# Value must be one of {'distributed', 'justify', 'general', 'right', 'centerContinuous', 'left', 'center', 'fill'}

name_style = Font(name = 'Arial Black', size = 18, color = '00000000')
name_align = Alignment(horizontal = 'centerContinuous', vertical = 'center')

team_style = Font(name = 'Arial Black', size = 12, color = '00000000')
team_align = Alignment(horizontal = 'centerContinuous', vertical = 'center')

merge_align = Alignment(horizontal = 'centerContinuous', vertical = 'center')

center_align = Alignment(horizontal = 'center', vertical = 'center')

for row in ws.iter_rows(min_row = 4, min_col = 2, max_row = 6, max_col = 2):
    for cell in row:
        cell.alignment = center_align
        
for row in ws.iter_rows(min_row = 4, min_col = 5, max_row = 6, max_col = 5):
    for cell in row:
        cell.alignment = center_align

for row in ws.iter_rows(min_row = 5, min_col = 8, max_row = 7, max_col = 8):
    for cell in row:
        cell.alignment = center_align

        
        
ws.merge_cells('A2:H2')
a2 = ws['A2']
a2.font = name_style
a2.alignment = name_align

ws.merge_cells('A3:E3')
a3 = ws['A3']
a3.font = team_style
a3.alignment = team_align

ws.merge_cells('A7:B7')
a7 = ws['A7']
a7.alignment = merge_align

ws.merge_cells('A8:B8')
a8 = ws['A8']
a8.alignment = merge_align

ws.merge_cells('A9:B9')
a9 = ws['A9']
a9.alignment = merge_align

ws.merge_cells('C7:D7')
c7 = ws['C7']
c7.alignment = merge_align

ws.merge_cells('C8:D8')
c8 = ws['C8']
c8.alignment = merge_align

ws.merge_cells('C9:D9')
c9 = ws['C9']
c9.alignment = merge_align




ws['A2'] = player[0]['nameFirstLast']
ws['A3'] = player_team[0]['name']
ws['A4'] = 'Number'
ws['B4'] = bio_df._get_value(0,'primaryNumber')
ws['A5'] = 'Position'
ws['B5'] = bio_df._get_value(0,'primaryPosition.abbreviation')
ws['D4'] = 'Bats'
ws['E4'] = bio_df._get_value(0,'batSide.code')
ws['D5'] = 'Throws'
ws['E5'] = bio_df._get_value(0,'pitchHand.code')
ws['A6'] = 'Height'
ws['B6'] = bio_df._get_value(0,'height')
ws['D6'] = 'Weight'
ws['E6'] = bio_df._get_value(0,'weight')
ws['A7'] = 'Date of Birth'
ws['C7'] = bio_df._get_value(0,'birthDate')
ws['A8'] = 'Place of Birth'
ws['C8'] = birthplace
ws['A9'] = 'MLB Debut'
ws['C9'] = player[0]['mlbDebutDate']
ws['A10'] = 'Awards'

ws['G4'] = 'Year Drafted'
ws['G5'] = 'Round'
ws['G6'] = 'Pick'
ws['G7'] = 'Overall Pick'
ws['G8'] = 'School'

try:
    ws['H4'] = dr[0]['draftYear']
    ws['H5'] = dr[0]['drafts'][0]['pickRound']
    ws['H6'] = dr[0]['drafts'][0]['roundPickNumber']
    ws['H7'] = dr[0]['drafts'][0]['pickNumber']
    ws['H8'] = dr[0]['drafts'][0]['school']['name']
    
except:
    ws['H4'] = 'Did not Participate in Draft'
    

    
if pd.DataFrame(player_awards_df).empty:
    ws['A11'] = 'No Awards won so far'

else:
    for r in dataframe_to_rows(player_awards_df, index = False, header = True):
        ws.append(r)
    

ws.append(['Player Season Stats'])

try:
    if stats_type == 'hitting':
        for r in dataframe_to_rows(player_stats_hitting, index = False, header = True):
            ws.append(r)
    
        
    else:
        
        for r in dataframe_to_rows(player_stats_pitching, index = False, header = True):
            ws.append(r)
    
    
except:
    print('No available season stats for that category for the selected player')

    
ws.append(['Player Career Stats'])

try:
    if stats_type == 'hitting':
        for r in dataframe_to_rows(player_career_hitting, index = False, header = True):
            ws.append(r)
    
        
    else:
        
        for r in dataframe_to_rows(player_career_pitching, index = False, header = True):
            ws.append(r)
    
    
except:
    print('No available career stats for that category for the selected player')


wb.save('Baseball Card.xlsx')
